In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [3]:
# 필요 라이브러리 임트트
import os
from pydantic import BaseModel
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

In [5]:
################################ 문서 검색 RAG 정의##############################################
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# --- 문서 로드 및 전처리 ---
loader = PyMuPDFLoader("stock_report/[삼성전자]분기보고서(2024.11.14).pdf")
docs = loader.load()

## : 문서 분할(Split Documents) <-----------추후 문서 제목 단위 분할로 변경 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

## 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 벡터스토어 생성
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings, persist_directory="stock_report/chroma_db")

# 5. 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 문서 검색 도구 생성
from langchain.tools.retriever import create_retriever_tool

# 해당 툴을 정의하는 것.
# 이를 이용하여 LLM에 해당 툴을 결합시킬 수 있음.
retriever_tool = create_retriever_tool(
    retriever,
    name="retriever_tool",   # 툴의 이름 (Agent가 호출할 때 사용)
    description="반기보고서 내용을 검색하여 제공합니다."  # 툴의 용도 설명
)

# 해당 툴 노드를 정의하는 것.
# Graph에서 사용하기 위해 Node로 만들 필요가 있음.
from langgraph.prebuilt import ToolNode

retrieve = ToolNode([retriever_tool])

In [6]:
################################# 웹 검색 및 날짜 확인 도구 ######################################
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults
from datetime import datetime
from langchain.agents import Tool
import os

# 웹검색 도구 생성
web_search = TavilySearchResults(max_results=3)
# 해당 함수의 경우에는 재검색이 요청된 경우에 사용하도록 한다.
web_search_retry = TavilySearchResults(max_results=5)

# 오늘 날짜 확인
def get_today_tool():
    return datetime.today().strftime('%Y-%m-%d')

today_tool = Tool(
    name="Get Today",
    func=get_today_tool,
    description="Returns today's date in YYYY-MM-DD format",
    verbose=True
)

# 생성된 도구 목록
tools = [web_search, web_search_retry, today_tool, retriever_tool]

In [25]:
# 노드 1-1 생성된 툴들에 대하여 툴노드 정의
from langgraph.prebuilt import ToolNode

tool_nodes = ToolNode(tools)

In [8]:
# 사용자 입력값 스테이트
class InputState(TypedDict):
    start_input: str

# 1 User Input 노드 
def user_input_node(state: InputState):
    print("================================= cal stock =================================")
    print("주식가치를 계산합니다. 궁금하신 주식명을 입력해주세요.")
    # time.sleep(1)
    user_input = input("User: ")
    
    return {"user_input": user_input}

In [9]:
# 주식 가치 정보 데이터 저장 스테이트
class StockValueCalState(TypedDict):
    user_input: str
    stock_value_dict: dict

In [10]:
# llm 설계
class Stock_value_output(TypedDict):
    """
    Sturctured_output을 생성하기위한 클래스
    """
    Net_Income: Annotated[str, ..., "순이익"]
    Shares_Outstanding: Annotated[str, ..., "발행주식수"]
    Stock_Price: Annotated[str, ..., "주가"]
    Book_Value: Annotated[str, ..., "자기자본"]
    FCF: Annotated[str, ..., "자유현금흐름"]
    EBIT: Annotated[str, ..., "영업이익"]
    WACC: Annotated[str, ..., "할인율"]
    PFCF: Annotated[str, ..., "미래 현금흐름"]
    Growth_Rate: Annotated[str, ..., "성장률"]
    DPS: Annotated[str, ..., "주당배당금"]
    ROE: Annotated[str, ..., "자기자본이익률"]
    PER: Annotated[str, ..., "주가수익비율"]
    PBR: Annotated[str, ..., "주가순자산비율"]
    DCF: Annotated[str, ..., "할인현금흐름"]

stock_value_model = ChatOpenAI(model="gpt-4o-mini")
stock_value_model = stock_value_model.with_structured_output(Stock_value_output)

In [20]:
# 2 stock value calculation 노드

def stock_value_calculation_node(state: StockValueCalState):
    messages = [
        ("system", """
        당신은 유능한 금융전문가 입니다.\n
        user input 을 받아서 관련 주식명을 추출하고, 해당 주식의 가치를 평가하여 답변하여야 합니다. \n
        주식 가치를 평가하기 위해 추출한 주식명의 'Net_Income', 'Shares_Outstanding', 'Stock_Price', 'Book_Value', 'FCF', 'EBIT', 'WACC', 'PFCF', 'Growth_Rate', 'DPS', 'ROE'를 retrieve Tool을 활용하여 값을 작성하시오.
        값을 찾지 못한 경우 빈 값으로 제출하시오.
        """),
        ("human", state['user_input'])
    ]
    response = tool_nodes.invoke(messages)
    
    print("================================= stock value calculation Setup =================================")
    print(f"입력된 정보:{state['user_input']}")
    print(f"순이익: {response['Net_Income']}")
    print(f"발행주식수: {response['Shares_Outstanding']}")
    print(f"주가: {response['Stock_Price']}")
    print(f"자기자본: {response['Book_Value']}")
    print(f"자유현금흐름: {response['FCF']}")
    print(f"영업이익: {response['EBIT']}")
    print(f"할인율: {response['WACC']}")
    print(f"미래 현금흐름: {response['PFCF']}")
    print(f"성장률: {response['Growth_Rate']}")
    print(f"주당배당금: {response['DPS']}")
    print(f"자기자본이익률: {response['ROE']}")
    print(f"주가수익비율: {response['PER']}")
    print(f"주가순자산비율: {response['PBR']}")
    print(f"할인현금흐름: {response['DCF']}")
    print(f"배당할인모형: {response['DDM']}")
    
    return {"stock_value_dict": response}

In [22]:
# 그래프 전체적으로 사용할 State를 정의
class OverallState(TypedDict):
    user_input: str
    messages: Annotated[list, add_messages]
    stock_value_dict: dict

# 그래프를 만드는 builder를 정의. input을 지정해주지 않으면 OverallState를 START에서 Input으로 요구하게 됨.
graph_builder = StateGraph(OverallState, input=InputState)

# 그래프의 Node를 추가함. 노드의 이름과 노드 함수를 인자로 받음.
graph_builder.add_node("User Input", user_input_node)
graph_builder.add_node("Stock Value Calculation Setup", stock_value_calculation_node)

# 그래프의 Edge를 추가함. 시작과 끝은 항상 START에서 END로 가야함
graph_builder.add_edge(START, "User Input")
graph_builder.add_edge("User Input", "Stock Value Calculation Setup")
graph_builder.add_edge("Stock Value Calculation Setup", END) # 노드에 순서에 맞추어 엣지를 조금 변경하자.

# 해당 그래프를 컴파일
graph = graph_builder.compile()

# 해당 그래프의 도식을 그려서 저장
with open("graph_output1.png", "wb") as f:
    f.write(graph.get_graph().draw_mermaid_png())

config = {"configurable": {"thread_id": "1"}}

# 그래프 호출. 아까 이야기 했듯 start_input에는 아무것도 입력하지 않음.
# 여기서도 마찬가지로 dict형태의 입력이 요구됨.
graph.invoke({"start_input": ""})

================================= cal stock =================================
주식가치를 계산합니다. 궁금하신 주식명을 입력해주세요.


User:  삼성전자


ValidationError: 1 validation error for RetrieverInput
  Input should be a valid dictionary or instance of RetrieverInput [type=model_type, input_value=[('system', "\n        ...human', '삼성전자')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type